<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Scraping Sephora website with Beautifulsoup


<img src="https://i.insider.com/5838406765edfed40a8b49b1?width=2500&format=jpeg&auto=webp" style="height: 400px; width: 1000px">


Name: Raghad Alharbi

---

- Link to dataset: https://www.kaggle.com/raghadalharbi/all-products-available-on-sephora-website
- Link to starter kernel: https://www.kaggle.com/raghadalharbi/sephora-starter-code
- Second starter kernel: https://www.kaggle.com/kerneler/starter-sephora-website-779abf66-7

In [1]:
import requests
import re  # Regular expression regex
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

BOLD = '\033[1m'
END = '\033[0m'

## Getting brands links from Sephora website

In [2]:
# 
brand_list_site= 'https://www.sephora.com/brands-list'
brand_list_request = requests.get(brand_list_site)
brand_list_html = brand_list_request.text

# getting beautiful soup for the brand list webpage
brand_list_soup = BeautifulSoup(brand_list_html,'html.parser')
brand_list=brand_list_soup.find_all('li',class_="css-15po5p8")

brand_list_URL = []
brand_list_name = []
for brand in brand_list:
    a = brand.find("a")
    brand_list_name.append(a.text[:-1])# I had to add [:-1] to remove the special characters at the end of the name 
    brand_list_URL.append('https://www.sephora.com'+ a['href']+'?pageSize=300') # get the full URL 
    
print("Number of brands in Sephora website: " , len(brand_list_name))   

Number of brands in Sephora website:  336


## Getting the number of items in each brand

In [108]:
def get_no_of_items_in_brands ():
    driver = webdriver.Chrome(executable_path='chromedriver/chromedriver')

    brand_list_site= 'https://www.sephora.com/brands-list'
    brand_list_request = requests.get(brand_list_site)
    brand_list_html = brand_list_request.text

    # getting beautiful soup for the brand list webpage
    brand_list_soup = BeautifulSoup(brand_list_html,'html.parser')
    brand_list=brand_list_soup.find_all('li',class_="css-15po5p8")

    brand_list_URL = []
    brand_list_name = []
    brand_list_no_of_items =[]

    print('Staring scrapping Sephora website brand by brand ..')
    print("Number of brands in Sephora website: " , len(brand_list)) 
    for brand in brand_list:
        a = brand.find("a")
        brand_name = a.text[:-1] # I had to add [:-1] to remove the special characters at the end of the name 
        print(BOLD , 'Brand name:', brand_name , END)
        brand_URL = 'https://www.sephora.com'+ a['href']
        print('Brand URL:', brand_URL)
        brand_list_name.append(brand_name)
        brand_list_URL.append(brand_URL) 
        
        driver.get(brand_URL)
        sleep(3) # Delay execution for a given number of seconds to allow the page to be loaded.
        
        brand_site_html  = driver.page_source # property/attribute -- Gets the source of the current page.
        brand_site_soup = BeautifulSoup(brand_site_html,'html.parser')
        print('Fininding all div objects in brand : ', brand_name)
        no_of_item=brand_site_soup.find('h2',class_="css-fgy0ne").find('span').text.split(' ')[0]
        brand_list_no_of_items.append(no_of_item)
        print('Number of items in brand ( ',brand_name ,' )= ',no_of_item )

    driver.close() # close the opened page once finish (if you need to download more pages, no need to close the driver)    
    #print("Number of brands in Sephora website: " , len(brand_list_name))  
    #print("Number of items in Sephora website: " , sum(no_of_item))  
    
    brand_df=pd.DataFrame({
                'brand_name':brand_list_name,
                'brand_url':brand_list_URL,
                'No_of_items':brand_list_no_of_items,
                })

    return brand_df

In [4]:
#brand_df = get_no_of_items_in_brands ()
#brand_df.head()
#brand_df.to_csv('sephora_brands_info.csv')

In [5]:
brand_df = pd.read_csv("sephora_brands_info.csv").drop('Unnamed: 0', axis = 1)
brand_df['No_of_items'] =brand_df['No_of_items'].astype(int)
brand_df.sort_values(by=['No_of_items'], ascending=False).head()  

,brand_name,brand_url,No_of_items
271,SEPHORA COLLECTION,https://www.sephora.com/brand/sephora-collection,500
59,CLINIQUE,https://www.sephora.com/brand/clinique,234
298,tarte,https://www.sephora.com/brand/tarte,172
305,TOM FORD,https://www.sephora.com/brand/tom-ford,161
77,Dior,https://www.sephora.com/brand/dior,145


From the the dataframe above, we can see the brands with the most number of items.
- SEPHORA COLLECTION: we know that there are 300 items in each page, so we need to go to the second page manually, as the next page button does not work. Also, we need to increase the scrolls to be able to get divs of items at the bottom of the page.

## Getting item URLs from all brands

In [11]:
# create browser obj
driver = webdriver.Chrome(executable_path='chromedriver/chromedriver')

brand_list_site= 'https://www.sephora.com/brands-list'
brand_list_request = requests.get(brand_list_site)
brand_list_html = brand_list_request.text

# getting beautiful soup for the brand list webpage
brand_list_soup = BeautifulSoup(brand_list_html,'html.parser')
brand_list=brand_list_soup.find_all('li',class_="css-15po5p8")

brand_list_URL = []
brand_list_name = []
item_list_id = []
item_list_name = []
item_list_URL = []

print('Staring scrapping Sephora website brand by brand ..')
print("Number of brands in Sephora website: " , len(brand_list)) 
for brand in brand_list:
    a = brand.find("a")
    brand_name = a.text[:-1] # I had to add [:-1] to remove the special characters at the end of the name 
    print(BOLD , 'Brand name:', brand_name , END)
    brand_URL = 'https://www.sephora.com' + a['href'] + '?pageSize=300'
    print('Brand URL:', brand_URL)
    driver.get(brand_URL)
    sleep(3) # Delay execution for a given number of seconds to allow the page to be loaded.

    #**********************************************************************************************************#
    ################################## Start of the scroll page code ###########################################
    elem = driver.find_element_by_tag_name("body")

    no_of_pagedowns = 50

    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        sleep(0.2)
        no_of_pagedowns-=1
    ################################## End of the scroll page code #############################################
    #**********************************************************************************************************#

    brand_site_html  = driver.page_source # property/attribute -- Gets the source of the current page.
    brand_site_soup = BeautifulSoup(brand_site_html,'html.parser')
    print('Fininding all div objects in brand : ', brand_name)
    item_list=brand_site_soup.find_all('div',class_="css-12egk0t")
    print('Number of items in brand ( ',brand_name ,' )= ',len (item_list) )

    for item in item_list:
        brand_list_name.append(brand_name)
        brand_list_URL.append(brand_URL) # get the full URL 
        a = item.find('a')
        #item_list_id.append(a['data-uid'])
        item_list_name.append(a['aria-label'])
        item_list_URL.append('https://www.sephora.com'+ a['href']) # get the full URL 
        #print('ID =', a['data-uid'])
        #print('Name=',  a['aria-label'])
        #print('URL = ', 'https://www.sephora.com'+ a['href'] )
    
    
    print("Number of items in brand: " , len(item_list))   
    print("Number of items in URL List:  " , len(item_list_URL))
        
driver.close() # close the opened page once finish (if you need to download more pages, no need to close the driver)    
print("Number of brands in Sephora website: " , len(brand_list))  
print("Number of items in Sephora website: " , len(item_list_URL)) 

item_url_df=pd.DataFrame({# 'item_id': item_list_id,
                          'item_name': item_list_name,
                          'item_url':item_list_URL,
                          'brand_name':brand_list_name,
                          'brand_url':brand_list_URL,
                         })

Staring scrapping Sephora website brand by brand ..
Number of brands in Sephora website:  335
 Brand name: Acqua Di Parma 
Brand URL: https://www.sephora.com/brand/acqua-di-parma?pageSize=300
Fininding all div objects in brand :  Acqua Di Parma
Number of items in brand (  Acqua Di Parma  )=  35
Number of items in brand:  35
Number of items in URL List:   35
 Brand name: AERIN 
Brand URL: https://www.sephora.com/brand/aerin-perfume?pageSize=300
Fininding all div objects in brand :  AERIN
Number of items in brand (  AERIN  )=  22
Number of items in brand:  22
Number of items in URL List:   57
 Brand name: Aether Beauty 
Brand URL: https://www.sephora.com/brand/aether-beauty?pageSize=300
Fininding all div objects in brand :  Aether Beauty
Number of items in brand (  Aether Beauty  )=  6
Number of items in brand:  6
Number of items in URL List:   63
 Brand name: Algenist 
Brand URL: https://www.sephora.com/brand/algenist?pageSize=300
Fininding all div objects in brand :  Algenist
Number of

In [12]:
#item_url_df.to_csv('sephora_item_urls.csv')

### Maniually adding the items in the second page for the brand: Sephora Collection

In [110]:
# create browser obj
driver = webdriver.Chrome(executable_path='chromedriver/chromedriver')
brand_name = 'SEPHORA COLLECTION'
brand_URL = 'https://www.sephora.com' + '/sephora-collection' + '?pageSize=300' + '&currentPage=2'
print('Brand URL:', brand_URL)
driver.get(brand_URL)
sleep(3) # Delay execution for a given number of seconds to allow the page to be loaded.


sephora_list_URL = []
sephora_list_name = []

sephora_item_list_name = []
sephora_item_list_URL = []

#item_list_URL =[]
#**********************************************************************************************************#
################################## Start of the scroll page code ###########################################
elem = driver.find_element_by_tag_name("body")

no_of_pagedowns = 50

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    sleep(0.2)
    no_of_pagedowns-=1
################################## End of the scroll page code #############################################
#**********************************************************************************************************#

brand_site_html  = driver.page_source # property/attribute -- Gets the source of the current page.
brand_site_soup = BeautifulSoup(brand_site_html,'html.parser')
print('Fininding all div objects in brand : ', brand_name)
item_list=brand_site_soup.find_all('div',class_="css-12egk0t")
print('Number of items in brand ( ',brand_name ,' )= ',len (item_list) )

for item in item_list:
    sephora_list_name.append(brand_name)
    sephora_list_URL.append(brand_URL) # get the full URL 
    a = item.find("a")
    sephora_item_list_name.append(a['aria-label'])
    item_url = 'https://www.sephora.com'+ a['href']
    sephora_item_list_URL.append(item_url) # get the full URL 
    #print(item_url)
sleep(0.3)

# close the opened page once finish (if you need to download more pages, no need to close the driver)  
driver.close() 
print("Number of items in brand: " , len(item_list))   
print("Number of items in URL List:  " , len(item_list_URL)) 


sephora_item_url_df=pd.DataFrame({# 'item_id': item_list_id,
                          'item_name': sephora_item_list_name,
                          'item_url':sephora_item_list_URL,
                          'brand_name':sephora_list_name,
                          'brand_url':sephora_list_URL,
                         })

Brand URL: https://www.sephora.com/sephora-collection?pageSize=300&currentPage=2
Fininding all div objects in brand :  SEPHORA COLLECTION
Number of items in brand (  SEPHORA COLLECTION  )=  204
Number of items in brand:  204
Number of items in URL List:   9192


In [35]:
# Appending the items from the second page to the full dataframe and saving it to a csv file
#item_url_df = item_url_df.append(sephora_item_url_df , ignore_index=True)
#item_url_df.to_csv('sephora_item_urls.csv')
item_url_df.tail(2)

,item_name,item_url,brand_name,brand_url
9190,SEPHORA COLLECTION Gift Card,https://www.sephora.com/product/gift-card-P370...,SEPHORA COLLECTION,https://www.sephora.com/sephora-collection?pag...
9191,SEPHORA COLLECTION Happy Birthday Gift Card,https://www.sephora.com/product/happy-birthday...,SEPHORA COLLECTION,https://www.sephora.com/sephora-collection?pag...


In [36]:
item_url_df = pd.read_csv("sephora_item_urls.csv").drop('Unnamed: 0', axis = 1)
item_url_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9192 entries, 0 to 9191
Data columns (total 4 columns):
item_name     9192 non-null object
item_url      9192 non-null object
brand_name    9192 non-null object
brand_url     9192 non-null object
dtypes: object(4)
memory usage: 287.4+ KB


## Getting data for each product from its webpage

In [194]:
item_list_URL = item_url_df['item_url'].tolist()
print("Starting processing all of items in Sephora website... ")
print("Number of items: ", len(item_list_URL))

# create browser obj
driver = webdriver.Chrome(executable_path='chromedriver/chromedriver')

item_name_list = []
item_category_list =[]
item_brand_list = []
item_id_list =[]
item_size_list = []
item_rating_list = []
item_number_of_reviews_list = []
item_love_list = []
item_price_list = []
item_value_price_list = []
MarketingFlags_list = []
MarketingFlags_content_list = []
item_options_list = []
item_details_list = []
item_how_to_use_list = []
item_ingredients_list = []
count = 0
for item_URL in item_list_URL:
    
    driver.get(item_URL)
    item_site_html  = driver.page_source # property/attribute -- Gets the source of the current page.
    item_site_soup = BeautifulSoup(item_site_html,'html.parser')
    
    print('Item n. ',count , ' URL:',item_URL )
######################################################################################### item_name    
    try:
        item_name = item_site_soup.find('span', class_='css-euydo4').text
    
    except:
        item_name = None
    
    #print('Item Name:',item_name )
    item_name_list.append(item_name)
######################################################################################### item_category      
    try:
        if item_site_soup.find('a', class_='css-lrl8sh') != None:
            item_category = item_site_soup.find('a', class_='css-lrl8sh').text
            
        else:
            item_category = None
           
    except:
        item_category = None
    
    #print('Item category:',item_category )
    item_category_list.append(item_category)    
#########################################################################################   item_brand
    try:
        item_brand = item_site_soup.find('span', class_='css-0').text
    
    except:
        item_brand = None
    
    #print('Item brand:',item_brand )
    item_brand_list.append(item_brand)
#########################################################################################    item_id
    try:
        item_id = item_site_soup.find('div', class_='css-v7k1z0').text.split('ITEM')[1]
    
    except:
        item_id = None
    
    #print('Item ID:',item_id )
    item_id_list.append(item_id)
#########################################################################################    item_size
    try:
        item_size = item_site_soup.find('div', class_='css-v7k1z0').text.split('ITEM')[0]
    
    except:
        item_size = None
    
    #print('Item size:',item_size )    
    item_size_list.append(item_size)
#########################################################################################   item_rating 
    try:
        item_rating = item_site_soup.find('div', class_="css-ychh9y")['aria-label']
    
    except:
        item_rating = None
    
    #print('Item rating:',item_rating )
    item_rating_list.append(item_rating)
#########################################################################################    item_number_of_reviews
    try:
        item_number_of_reviews = item_site_soup.find('span', class_='css-2rg6q7').text
    
    except:
        item_number_of_reviews = None
    
    #print('Item number of reviews:',item_number_of_reviews )
    item_number_of_reviews_list.append(item_number_of_reviews)
#########################################################################################    item_love
    try: 
        item_love = item_site_soup.find('div', class_='css-1x6b5cp').find('div', class_='css-jtf0an').find('span', class_='css-2rg6q7').text
    
    except:
        item_love = None
    
    #print('Item love:',item_love )
    item_love_list.append(item_love)
#########################################################################################    item_price
    try:
        item_price = item_site_soup.find('div', class_='css-slwsq8').find('span').text
    
    except:
        item_price = None 
    #print('Item price:',item_price )
    item_price_list.append(item_price)
#########################################################################################   item_value_price 
    try:
        if  item_site_soup.find('div', class_='css-acn7bs') != None:
            item_value_price = item_site_soup.find('div', class_='css-acn7bs').text
            
        else:
            item_value_price = None
            
    except:  
        item_value_price = None
    
    #print('Item value price:',item_value_price )    
    item_value_price_list.append(item_value_price)    
######################################################################################### MarketingFlags       
    try:
        if item_site_soup.find('div', class_='css-x8fq2p') != None:
            MarketingFlags  = item_site_soup.find('div', class_='css-x8fq2p').text != None
            

            if MarketingFlags == True:
                MarketingFlags_content = item_site_soup.find('div', class_='css-x8fq2p').text
            else:  
                MarketingFlags_content = None
                           
        else: 
            MarketingFlags  = False

            MarketingFlags_content = None
    except: 
        MarketingFlags  = False
        MarketingFlags_content = None
    
    #print('MarketingFlags:',MarketingFlags )
    #print('Content:',MarketingFlags_content )
    MarketingFlags_list.append(MarketingFlags)
    MarketingFlags_content_list.append(MarketingFlags_content)
    
#########################################################################################     item_options 
    try:
        item_options = ''
        options = item_site_soup.find_all('div', class_='css-5jqxch',attrs= {'data-comp': 'ProductSwatchItem '})
        for option in options:
             item_options=item_options+' - ' + (option.find('button')['aria-label'])
        item_options = item_options.replace('- Selected','')
    
    except:
        item_options = None
   
    #print('Item options: ', item_options);print()
    item_options_list.append(item_options)
#########################################################################################    item_details
    try:
        details_key = ''
        for i in item_site_soup.find_all('button', class_=["css-18ih2xz", "css-1h2tppq", 'css-kg29xc ']) :
            if (i.text == 'Details'):
                details_key = i['id']
                   
        item_details = item_site_soup.find('div', class_='css-1rny024',attrs= {'aria-labelledby': details_key}).text.replace(',', '-')
        
    except:
        item_details = None
    #print('Item details:',item_details );print()
    item_details_list.append(item_details)
#########################################################################################   item_how_to_use 
    try:
        how_key = ''
        for i in item_site_soup.find_all('button', class_=["css-18ih2xz", "css-1h2tppq", 'css-kg29xc ']) :
            if (i.text == 'How to Use'):
                how_key = i['id']
            
        item_how_to_use = item_site_soup.find('div', class_='css-1rny024' ,attrs= {'aria-labelledby': how_key}).text.replace(',', '-')
    
    except:
        item_how_to_use = None
    
    #print('Item how to use:',item_how_to_use );print()
    item_how_to_use_list.append(item_how_to_use)
######################################################################################### item_ingredients
    try:
        ingredients_key = ''
        for i in item_site_soup.find_all('button', class_=["css-18ih2xz", "css-1h2tppq", 'css-kg29xc ']) :
            if (i.text == 'Ingredients'):
                ingredients_key = i['id']

        item_ingredients = item_site_soup.find('div', class_='css-1rny024' ,attrs= {'aria-labelledby': ingredients_key}).text.replace(',', '-')
     
    except:
        item_ingredients = None
    #print('Item ngredients:',item_ingredients )  ;print()  
    item_ingredients_list.append(item_ingredients)
    count+=1
######################################################################################### end ..


driver.close() # close the opened page once finish (if you need to download more pages, no need to close the driver)

full_df=pd.DataFrame({ 'id' : item_id_list,
                      'name' :item_name_list,
                      'category': item_category_list,
                      'brand': item_brand_list,
                      'size': item_size_list,
                      'rating' :item_rating_list,
                      'number_of_reviews': item_number_of_reviews_list,
                      'love': item_love_list,
                      'price':item_price_list,
                      'value_price':item_value_price_list,
                      'URL': item_list_URL,
                      'MarketingFlags': MarketingFlags_list,
                      'MarketingFlags_content' :MarketingFlags_content_list,
                      'options':item_options_list,
                      'details': item_details_list,
                      'how_to_use': item_how_to_use_list,
                      'ingredients': item_ingredients_list,
                     })

#full_df.to_csv('sephora_all_items_new.csv')

Starting processing all of items in Sephora website... 
Number of items:  9192
Item n.  0  URL: https://www.sephora.com/product/blu-mediterraneo-minature-set-P443401?icid2=products grid:p443401
Item n.  1  URL: https://www.sephora.com/product/colonia-P163604?icid2=products grid:p163604
Item n.  2  URL: https://www.sephora.com/product/blu-mediterraneo-arancia-di-capri-P375388?icid2=products grid:p375388
Item n.  3  URL: https://www.sephora.com/product/blu-mediterraneo-mirto-di-panarea-P307804?icid2=products grid:p307804
Item n.  4  URL: https://www.sephora.com/product/colonia-miniature-set-P443400?icid2=products grid:p443400
Item n.  5  URL: https://www.sephora.com/product/blu-mediterraneo-fico-di-amalfi-P307801?icid2=products grid:p307801
Item n.  6  URL: https://www.sephora.com/product/rosa-nobile-P388670?icid2=products grid:p388670
Item n.  7  URL: https://www.sephora.com/product/colonia-essenza-P269110?icid2=products grid:p269110
Item n.  8  URL: https://www.sephora.com/product/peon

In [3]:
#full_df.to_csv('sephora_all_items_new.csv')
df = pd.read_csv("sephora_all_items_new.csv").drop('Unnamed: 0', axis = 1)
df.rename(columns = {'brand':'name1', 'name':'brand'}, inplace = True) 
df.rename(columns = {'name1':'name'}, inplace = True) 

## switching the name and brand columns for some of the prands, as it was switched in the website.

In [4]:

df.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,URL,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients
0,2218774.0,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,SIZE 5 x 0.16oz/5mL•,4 stars,4 reviews,3002 loves,$66.00,($75.00 value),https://www.sephora.com/product/blu-mediterran...,True,online only,NaN,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...
1,2044816.0,Acqua Di Parma,Cologne,Colonia,SIZE 0.7 oz/ 20 mL•,4.5 stars,76 reviews,2.7K loves,$66.00,NaN,https://www.sephora.com/product/colonia-P16360...,True,online only,- 0.7 oz/ 20 mL Spray - 1.7 oz/ 50 mL Eau d...,An elegant timeless scent filled with a fresh-...,NaN,NaN
2,1417567.0,Acqua Di Parma,Perfume,Arancia di Capri,SIZE 5 oz/ 148 mL•,4.5 stars,26 reviews,2.6K loves,$180.00,NaN,https://www.sephora.com/product/blu-mediterran...,True,online only,- 1oz/30mL Eau de Toilette - 2.5 oz/ 74 mL E...,Fragrance Family: Fresh Scent Type: Fresh Citr...,NaN,Alcohol Denat.- Water- Fragrance- Limonene- Li...
3,1417617.0,Acqua Di Parma,Perfume,Mirto di Panarea,SIZE 2.5 oz/ 74 mL•,4.5 stars,23 reviews,2.9K loves,$120.00,NaN,https://www.sephora.com/product/blu-mediterran...,True,online only,- 1 oz/ 30 mL Eau de Toilette Spray - 2.5 oz/...,Panarea near Sicily is an an island suspended ...,NaN,NaN
4,2218766.0,Acqua Di Parma,Fragrance,Colonia Miniature Set,SIZE 5 x 0.16oz/5mL•,3.5 stars,2 reviews,943 loves,$72.00,($80.00 value),https://www.sephora.com/product/colonia-miniat...,True,online only,NaN,The Colonia Miniature Set comes in an iconic A...,Suggested Usage:-Fragrance is intensified by t...,Colonia: Alcohol Denat.- Water- Fragrance- Lim...


In [5]:
df.tail()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,URL,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients
9187,2208502.0,SEPHORA COLLECTION,Face Masks,The Rose Gold Mask,NaN,2 stars,15 reviews,6.2K loves,$6.00,NaN,https://www.sephora.com/product/the-rose-gold-...,True,limited edition · exclusive,NaN,What it is: A limited-edition- nurturing and h...,Suggested Usage:-Unfold the mask.-Apply the ma...,-Rose Quartz Extract: Hydrates dry skin. Aqua...
9188,2298909.0,SEPHORA COLLECTION,Lip Sets,Give Me Some Sugar Colorful Gloss Balm Set,SIZE 3 x 0.32 oz/ 9 g•,No stars,0 reviews,266 loves,$15.00,($27.00 value),https://www.sephora.com/product/sephora-collec...,True,exclusive,NaN,What it is: A set of three bestselling Colorfu...,Suggested Usage:-Apply directly to lips using ...,Colorful Gloss Balm Wanderlust: Hydrogenated P...
9189,2236750.0,SEPHORA COLLECTION,Tinted Moisturizer,Weekend Warrior Tone Up Cream,SIZE 0.946 oz/ 28 mL•,No stars,0 reviews,445 loves,$16.00,NaN,https://www.sephora.com/product/sephora-collec...,True,exclusive,NaN,What it is: A weightless complexion booster- i...,Suggested Usage:-Use this product as the last ...,Aqua (Water)- Dimethicone- Isohexadecane- Poly...
9190,50.0,SEPHORA COLLECTION,NaN,Gift Card,NaN,5 stars,46 reviews,NaN,$50.00,NaN,https://www.sephora.com/product/gift-card-P370...,False,NaN,NaN,What it is:- Available in denominations of $10...,NaN,NaN
9191,304.0,SEPHORA COLLECTION,NaN,Happy Birthday Gift Card,NaN,No stars,0 reviews,NaN,$50.00,NaN,https://www.sephora.com/product/happy-birthday...,False,NaN,NaN,What it is:\n- Available in denominations of $...,NaN,NaN


In [6]:
#pd.options.mode.chained_assignment = None
empty_url_list = df.loc[df['price'].isna(), 'URL'].tolist()
print("Starting processing all of items in Sephora website... ")
print("Number of items: ", len(empty_url_list))
count = 0
# create browser obj
driver = webdriver.Chrome(executable_path='chromedriver/chromedriver')


for item_URL in empty_url_list:
    
    driver.get(item_URL)
    item_site_html  = driver.page_source # property/attribute -- Gets the source of the current page.
    item_site_soup = BeautifulSoup(item_site_html,'html.parser')
    
    print('Item n. ',count , ' URL:',item_URL )
######################################################################################### item_name    
    try:
        item_name = item_site_soup.find('span', class_='css-euydo4').text
    
    except:
        item_name = None
    
    #print('Item Name:',item_name )
    df.loc[df['URL']== item_URL, 'brand'] = item_name
######################################################################################### item_category      
    try:
        if item_site_soup.find('a', class_='css-lrl8sh') != None:
            item_category = item_site_soup.find('a', class_='css-lrl8sh').text
            
        else:
            item_category = None
           
    except:
        item_category = None
    
    #print('Item category:',item_category )
    df.loc[df['URL']== item_URL, 'category'] =   item_category
#########################################################################################   item_brand
    try:
        item_brand = item_site_soup.find('span', class_='css-0').text
    
    except:
        item_brand = None
    
    #print('Item brand:',item_brand )
    df.loc[df['URL']== item_URL, 'name'] = item_brand
#########################################################################################    item_id
    try:
        item_id = item_site_soup.find('div', class_='css-v7k1z0').text.split('ITEM')[1]
    
    except:
        item_id = None
    
    #print('Item ID:',item_id )
    df.loc[df['URL']== item_URL, 'id'] = item_id
#########################################################################################    item_size
    try:
        item_size = item_site_soup.find('div', class_='css-v7k1z0').text.split('ITEM')[0]
    
    except:
        item_size = None
    
    #print('Item size:',item_size )    
    df.loc[df['URL']== item_URL, 'size'] = item_size
#########################################################################################   item_rating 
    try:
        item_rating = item_site_soup.find('div', class_="css-ychh9y")['aria-label']
    
    except:
        item_rating = None
    
    #print('Item rating:',item_rating )
    df.loc[df['URL']== item_URL, 'rating'] = item_rating
#########################################################################################    item_number_of_reviews
    try:
        item_number_of_reviews = item_site_soup.find('span', class_='css-2rg6q7').text
    
    except:
        item_number_of_reviews = None
    
    #print('Item number of reviews:',item_number_of_reviews )
    df.loc[df['URL']== item_URL, 'number_of_reviews'] = item_number_of_reviews
#########################################################################################    item_love
    try: 
        item_love = item_site_soup.find('div', class_='css-1x6b5cp').find('div', class_='css-jtf0an').find('span', class_='css-2rg6q7').text
    
    except:
        item_love = None
    
    #print('Item love:',item_love )
    df.loc[df['URL']== item_URL, 'love'] = item_love
#########################################################################################    item_price
    try:
        item_price = item_site_soup.find('div', class_='css-slwsq8').find('span').text
    
    except:
        item_price = None 
    #print('Item price:',item_price )
    df.loc[df['URL']== item_URL, 'price'] = item_price
#########################################################################################   item_value_price 
    try:
        if  item_site_soup.find('div', class_='css-acn7bs') != None:
            item_value_price = item_site_soup.find('div', class_='css-acn7bs').text
            
        else:
            item_value_price = None
            
    except:  
        item_value_price = None
    
    #print('Item value price:',item_value_price )    
    df.loc[df['URL']== item_URL, 'value_price'] = item_value_price
######################################################################################### MarketingFlags       
    try:
        if item_site_soup.find('div', class_='css-x8fq2p') != None:
            MarketingFlags  = item_site_soup.find('div', class_='css-x8fq2p').text != None
            

            if MarketingFlags == True:
                MarketingFlags_content = item_site_soup.find('div', class_='css-x8fq2p').text
            else:  
                MarketingFlags_content = None
                           
        else: 
            MarketingFlags  = False

            MarketingFlags_content = None
    except: 
        MarketingFlags  = False
        MarketingFlags_content = None
    
    #print('MarketingFlags:',MarketingFlags )
    #print('Content:',MarketingFlags_content )
    
    df.loc[df['URL']== item_URL, 'MarketingFlags'] = MarketingFlags
    df.loc[df['URL']== item_URL, 'MarketingFlags_content'] = MarketingFlags_content
#########################################################################################     item_options 
    try:
        item_options = ''
        options = item_site_soup.find_all('div', class_='css-5jqxch',attrs= {'data-comp': 'ProductSwatchItem '})
        for option in options:
             item_options=item_options+' - ' + (option.find('button')['aria-label'])
        item_options = item_options.replace('- Selected','')
    
    except:
        item_options = None
   
    #print('Item options: ', item_options);print()
    df.loc[df['URL']== item_URL, 'options'] = item_options
#########################################################################################    item_details
    try:
        details_key = ''
        for i in item_site_soup.find_all('button', class_=["css-18ih2xz", "css-1h2tppq", 'css-kg29xc ']) :
            if (i.text == 'Details'):
                details_key = i['id']
                   
        item_details = item_site_soup.find('div', class_='css-1rny024',attrs= {'aria-labelledby': details_key}).text.replace(',', '-')
        
    except:
        item_details = None
    #print('Item details:',item_details );print()
    df.loc[df['URL']== item_URL, 'details'] = item_details
#########################################################################################   item_how_to_use 
    try:
        how_key = ''
        for i in item_site_soup.find_all('button', class_=["css-18ih2xz", "css-1h2tppq", 'css-kg29xc ']) :
            if (i.text == 'How to Use'):
                how_key = i['id']
            
        item_how_to_use = item_site_soup.find('div', class_='css-1rny024' ,attrs= {'aria-labelledby': how_key}).text.replace(',', '-')
    
    except:
        item_how_to_use = None
    
    #print('Item how to use:',item_how_to_use );print()
    df.loc[df['URL']== item_URL, 'how_to_use'] = item_how_to_use
######################################################################################### item_ingredients
    try:
        ingredients_key = ''
        for i in item_site_soup.find_all('button', class_=["css-18ih2xz", "css-1h2tppq", 'css-kg29xc ']) :
            if (i.text == 'Ingredients'):
                ingredients_key = i['id']

        item_ingredients = item_site_soup.find('div', class_='css-1rny024' ,attrs= {'aria-labelledby': ingredients_key}).text.replace(',', '-')
     
    except:
        item_ingredients = None
    #print('Item ingredients:',item_ingredients )  ;print()      
    df.loc[df['URL']== item_URL, 'ingredients'] = item_ingredients
    
    count+=1
######################################################################################### end ..


driver.close() # close the opened page once finish (if you need to download more pages, no need to close the driver)


Starting processing all of items in Sephora website... 
Number of items:  86
Item n.  0  URL: https://www.sephora.com/product/shimmering-skin-perfector-spf-25-P375987?icid2=products grid:p375987
Item n.  1  URL: https://www.sephora.com/product/shimmering-skin-perfector-liquid-highlighter-mini-P422074?icid2=products grid:p422074
Item n.  2  URL: https://www.sephora.com/product/jessica-rabbit-compact-mirror-P451637?skuId=2287803&icid2=products grid:p451637
Item n.  3  URL: https://www.sephora.com/product/7-day-scrub-cream-rinse-off-formula-mini-P440311?icid2=products grid:p440311
Item n.  4  URL: https://www.sephora.com/product/beyond-perfecting-super-concealer-camouflage-24-hour-wear-apricot-color-corrector-P424829?icid2=products grid:p424829
Item n.  5  URL: https://www.sephora.com/product/superprimer-face-primer-colour-corrects-redness-P382502?icid2=products grid:p382502
Item n.  6  URL: https://www.sephora.com/product/clinique-detox-defend-P456432?icid2=products grid:p456432
Item n. 

In [7]:
#df.to_csv('sephora_all_items_new.csv')

In [281]:
df['name'].isna().sum()

8

In [287]:
df = df[df['price'].notna()]

In [306]:
df.head(2)

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,URL,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients
0,2.21877e+06,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,SIZE 5 x 0.16oz/5mL•,4 stars,4 reviews,3002 loves,$66.00,($75.00 value),https://www.sephora.com/product/blu-mediterran...,True,online only,NaN,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...
1,2.04482e+06,Acqua Di Parma,Cologne,Colonia,SIZE 0.7 oz/ 20 mL•,4.5 stars,76 reviews,2.7K loves,$66.00,NaN,https://www.sephora.com/product/colonia-P16360...,True,online only,- 0.7 oz/ 20 mL Spray - 1.7 oz/ 50 mL Eau d...,An elegant timeless scent filled with a fresh-...,NaN,NaN


In [343]:
df['rating']= df['rating'].str.split('stars', expand=True)[0]
df['number_of_reviews']= df['number_of_reviews'].str.split('reviews', expand=True)[0]
df['love']= df['love'].str.split('loves', expand=True)[0]
df['love']= df['love'].str.replace('K', '000').str.replace('.','')
df['price']= df['price'].str.replace('$', '')
df['value_price']= df['value_price'].str.replace('\(', '').str.replace('value\)', '').str.replace('$', '')
#df['number_of_items'] = df['size'].str.split('SIZE ', expand=True)[1]
#df['size'] = df['size'].str.split('SIZE ', expand=True)[1].str.split('x ', expand=True)[1]


0         0.16oz/5mL•
1                None
2                None
3                None
4         0.16oz/5mL•
            ...      
9187              NaN
9188    0.32 oz/ 9 g•
9189             None
9190              NaN
9191              NaN
Name: size, Length: 9168, dtype: object

In [339]:
df.head(10)

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,URL,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients,number_of_items
0,2.21877e+06,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,0.16oz/5mL•,4,4,3002,66.00,75.00,https://www.sephora.com/product/blu-mediterran...,True,online only,NaN,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...,5 x 0.16oz/5mL•
1,2.04482e+06,Acqua Di Parma,Cologne,Colonia,None,4.5,76,27000,66.00,NaN,https://www.sephora.com/product/colonia-P16360...,True,online only,- 0.7 oz/ 20 mL Spray - 1.7 oz/ 50 mL Eau d...,An elegant timeless scent filled with a fresh-...,NaN,NaN,0.7 oz/ 20 mL•
2,1.41757e+06,Acqua Di Parma,Perfume,Arancia di Capri,None,4.5,26,26000,180.00,NaN,https://www.sephora.com/product/blu-mediterran...,True,online only,- 1oz/30mL Eau de Toilette - 2.5 oz/ 74 mL E...,Fragrance Family: Fresh Scent Type: Fresh Citr...,NaN,Alcohol Denat.- Water- Fragrance- Limonene- Li...,5 oz/ 148 mL•
3,1.41762e+06,Acqua Di Parma,Perfume,Mirto di Panarea,None,4.5,23,29000,120.00,NaN,https://www.sephora.com/product/blu-mediterran...,True,online only,- 1 oz/ 30 mL Eau de Toilette Spray - 2.5 oz/...,Panarea near Sicily is an an island suspended ...,NaN,NaN,2.5 oz/ 74 mL•
4,2.21877e+06,Acqua Di Parma,Fragrance,Colonia Miniature Set,0.16oz/5mL•,3.5,2,943,72.00,80.00,https://www.sephora.com/product/colonia-miniat...,True,online only,NaN,The Colonia Miniature Set comes in an iconic A...,Suggested Usage:-Fragrance is intensified by t...,Colonia: Alcohol Denat.- Water- Fragrance- Lim...,5 x 0.16oz/5mL•
5,1.41761e+06,Acqua Di Parma,Perfume,Fico di Amalfi,None,4.5,79,26000,180.00,NaN,https://www.sephora.com/product/blu-mediterran...,True,online only,- 1oz/30mL Eau de Toilette - 2.5 oz/ 74 mL E...,Fragrance Family: Floral Scent Type: Fruity Fl...,NaN,NaN,5 oz/ 148 mL•
6,1.63883e+06,Acqua Di Parma,Perfume,Rosa Nobile,None,4.5,79,5000,210.00,NaN,https://www.sephora.com/product/rosa-nobile-P3...,True,online only,"- 0.7 oz, 20 mL Eau de Parfum Spray - 1.7 oz/...",Fragrance Family: Floral\n\nScent Type: Classi...,NaN,NaN,3.4 oz/ 101 mL•
7,1.28446e+06,Acqua Di Parma,Cologne,Colonia Essenza,None,5,13,719,120.00,NaN,https://www.sephora.com/product/colonia-essenz...,True,online only,- 1.7 oz/ 50 mL Eau de Cologne Spray - 3.4 o...,The newest Eau de Cologne from Acqua di Parma ...,NaN,Alcohol Denat.- Water- Fragrance- Limonene- Hy...,1.7 oz/ 50 mL•
8,2.22159e+06,Acqua Di Parma,Body Mist & Hair Mist,Peonia Nobile Hair Mist,None,4,5,800,58.00,NaN,https://www.sephora.com/product/peonia-nobile-...,True,online only,- 1.7oz/50mL Hair Mist - 0.70oz/20mL Eau de...,A new Le Nobili beauty ritual leaves your hair...,Suggested Usage:-Spray over your hair to leave...,Alcohol Denat.- Water- Fragrance- Polyquaterni...,1.7oz/50mL•
9,2.2216e+06,Acqua Di Parma,Perfume,Rosa Nobile Hair Mist,None,3,5,21000,58.00,NaN,https://www.sephora.com/product/rosa-nobile-ha...,True,exclusive · online only,- 1.7oz/50mL Hair Mist,Fragrance Family: Floral\n\nScent Type: Classi...,Suggested Usage:-Spray over your hair to leave...,NaN,1.7oz/50mL•
